In [ ]:
from pathlib import Path

import geopandas as gpd
from calculate_accessibility import determine_isochrones, initialize_valhalla
from common import load_stops

In [ ]:
CITY = "budapest"
VERSION = ""

In [3]:
Path(f"../output/{CITY}").mkdir(parents=True, exist_ok=True)

stops = load_stops(CITY)

actor = initialize_valhalla(CITY)

2025/03/27 12:19:27.987615 [INFO] Tile extract successfully loaded with tile count: 255
2025/03/27 12:19:27.987677 [WARN] (stat): /data/valhalla/traffic.tar No such file or directory
2025/03/27 12:19:27.987680 [WARN] Traffic tile extract could not be loaded


In [15]:
isochones = determine_isochrones(stops, actor)
isochones.to_csv(f"../output/{CITY}/isochrones.csv", index=False)

isochrones_gdf = gpd.GeoDataFrame(isochones, geometry="geometry", crs=4326)
isochrones_gdf.to_file(f"../output/{CITY}/isochrones.geojson")

In [ ]:
# isochones = pd.read_csv(f"../output/{CITY}/isochrones.csv")
# print(isochones.head(6).to_markdown(index=False))